In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.metrics import accuracy_score


In [2]:

# Load the breast cancer dataset
data = pd.read_csv(r'dataset\breast-cancer.csv')
data['diagnosis'] = data['diagnosis'].map({'M': 1, 'B': 0})

X = data

# Preprocess the data (scale it)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


FileNotFoundError: [Errno 2] No such file or directory: 'dataset\\breast-cancer.csv'

In [ ]:

# Prepare data for ELM
# Create labels for ELM training (1 for inliers, -1 for outliers)
Y = np.ones(len(X_scaled)) # All points are inliers

# Inject outliers into the dataset
def inject_outliers(X, num_outliers=5, factor=10):
    np.random.seed(42)  # For reproducibility
    indices = np.random.choice(len(X), num_outliers, replace=False)
    print(indices)
    for idx in indices:
        # Inject outlier by scaling the chosen points
        Y[idx] = -1
        X[idx] *= factor  # Multiply by a factor to create an outlier
    
    return X

# Create a copy of the original dataset to inject outliers
X_with_outliers = inject_outliers(X_scaled.copy(), num_outliers=10, factor=10)


[204  70 131 431 540 567 369  29  81 477]


In [ ]:

# Split the data into training and testing sets
X_train, X_test,y_train,y_test = train_test_split(X_with_outliers,Y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(455, 32) (114, 32) (455,) (114,)


In [ ]:
# Define the autoencoder architecture
input_dim = X_train.shape[1]
encoding_dim = 10  # Reduced dimension

# Input layer
input_layer = Input(shape=(input_dim,))
# Encoder layer
encoded = Dense(encoding_dim, activation='relu')(input_layer)
# Decoder layer
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# Autoencoder model
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, shuffle=True, validation_split=0.1)


Epoch 1/10
13/13 [==============================] - 2s 61ms/step - loss: 0.1060 - val_loss: 0.1109
Epoch 2/10
13/13 [==============================] - 0s 15ms/step - loss: 0.1001 - val_loss: 0.1047
Epoch 3/10
13/13 [==============================] - 0s 14ms/step - loss: 0.0939 - val_loss: 0.0976
Epoch 4/10
13/13 [==============================] - 0s 16ms/step - loss: 0.0867 - val_loss: 0.0895
Epoch 5/10
13/13 [==============================] - 0s 17ms/step - loss: 0.0788 - val_loss: 0.0806
Epoch 6/10
13/13 [==============================] - 0s 14ms/step - loss: 0.0704 - val_loss: 0.0715
Epoch 7/10
13/13 [==============================] - 0s 17ms/step - loss: 0.0625 - val_loss: 0.0631
Epoch 8/10
13/13 [==============================] - 0s 16ms/step - loss: 0.0557 - val_loss: 0.0554
Epoch 9/10
13/13 [==============================] - 0s 16ms/step - loss: 0.0498 - val_loss: 0.0488
Epoch 10/10
13/13 [==============================] - 0s 15ms/step - loss: 0.0451 - val_loss: 0.0440


In [ ]:
# # Get the reconstruction of the test set
# X_test_reconstructed = autoencoder.predict(X_test)

# # Calculate reconstruction error (mean squared error)
# reconstruction_error = np.mean(np.power(X_test - X_test_reconstructed, 2), axis=1)

In [ ]:
# # Set a threshold for outlier detection
# threshold = np.percentile(reconstruction_error, 95)  # 95th percentile
# outliers_detected = reconstruction_error > threshold

In [ ]:
# Initialize ELM Class
class ELM:
    def __init__(self, input_dim, num_hidden):
        self.input_dim = input_dim
        self.num_hidden = num_hidden
        self.weights = np.random.rand(input_dim, num_hidden)
        self.bias = np.random.rand(num_hidden)
        self.alpha = None

    def _activation(self, x):
        return np.maximum(0, x)  # ReLU activation

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        H = self._activation(np.dot(X, self.weights) + self.bias)
        self.alpha = np.linalg.pinv(H).dot(y)

    def predict(self, X):
        X = np.array(X)
        H = self._activation(np.dot(X, self.weights) + self.bias)
        return np.dot(H, self.alpha)


# # Prepare data for ELM
# # Create labels for ELM training (1 for inliers, -1 for outliers)
# y_train = np.ones(len(X_train))

# # Determine the mean and threshold
# mean_X_train = np.mean(X_train, axis=0)
# threshold = 1.5 * mean_X_train

# # Use a loop or boolean mask to mark outliers
# for i in range(len(X_train)):
#     if any(X_train[i] > threshold):
#         y_train[i] = -1  # Mark as outlier

# Initialize ELM
elm = ELM(input_dim=encoding_dim, num_hidden=20)


In [ ]:
# Use the encoder part of the autoencoder to reduce dimensions
encoder = Model(input_layer, encoded)
X_train_reduced = encoder.predict(X_train)
X_test_reduced = encoder.predict(X_test)

4/4 [==============================] - 0s 5ms/step


In [ ]:
print(X_train_reduced.shape, X_test_reduced.shape, y_train.shape)
# Fit the ELM model
elm.fit(X_train_reduced, y_train)

# Predict on the test set
y_pred = elm.predict(X_test_reduced)


(455, 10) (114, 10) (455,)


In [ ]:
# Convert predictions to binary (1 for inliers, -1 for outliers)
y_pred_binary = (y_pred >= 0).astype(int)  # Set a threshold to convert to binary

In [ ]:
# Actual outlier labels
# actual_outliers = np.zeros(len(X_test), dtype=int)
# actual_outliers[outliers_detected] = 1  # Label detected outliers as 1

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy of outlier detection:", accuracy)

Accuracy of outlier detection: 0.9122807017543859
